<a href="https://colab.research.google.com/github/ailab-nda/ML/blob/main/WebScraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Web スクレイピング

出典　https://code.dividable.net/tutorials/python-scraping-blog/

In [ ]:
import pandas as pd
import csv
import time
import random
import re
import json
from bs4 import BeautifulSoup
import requests

## WebページのHTMLを取得

In [ ]:
response = requests.get("https://honnobi.com/proverb-roland")
html_doc = response.text
soup = BeautifulSoup(html_doc, 'html.parser') 
#print(soup.prettify())

## 欲しい部分を抽出

In [ ]:
# ここはソースを見て必要なところを決める
tags = soup.find_all("strong")
for i in tags:
    print(i.text)

## ファイルに保存

In [ ]:
f = open("roland.txt","w")
for i in tags:
    f.write(i.text+"\n")
f.close()